In [29]:
using Pkg
# Pkg.activate("../../")
Pkg.activate("../../", io=devnull)
Pkg.update()

    Updating registry at `C:\Users\jakdar\.julia\registries\General.toml`
   Installed Opus_jll ─ v1.6.0+0
   Installed Revise ─── v3.13.0
    Updating `C:\Users\jakdar\uni\bachelorarbeit\VeryDiff\Project.toml`
  [295af30f] ↑ Revise v3.12.3 ⇒ v3.13.0
    Updating `C:\Users\jakdar\uni\bachelorarbeit\VeryDiff\Manifest.toml`
  [295af30f] ↑ Revise v3.12.3 ⇒ v3.13.0
  [91d4177d] ↑ Opus_jll v1.5.2+0 ⇒ v1.6.0+0
Precompiling project...
   5251.8 ms  ✓ Opus_jll
  11330.8 ms  ✓ Revise
   1523.7 ms  ✓ Revise → DistributedExt
   8191.6 ms  ✓ VeryDiff
  4 dependencies successfully precompiled in 95 seconds. 283 already precompiled.
  2 dependencies precompiled but different versions are currently loaded. Restart julia to access the new versions. Otherwise, loading dependents of these packages may trigger further precompilation to work with the unexpected versions.
  1 dependency had output during precompilation:
┌ Revise
│  ┌ Warning: precompile directive
│  │     precompile(Tuple{mbody.sig.paramet

In [30]:
using VeryDiff
using LinearAlgebra
using Plots

include("../util.jl")

RecipesBase.apply_recipe

In [31]:
struct Result
    net :: String
    spec :: String
    status :: VeryDiff.VerificationStatus
    runtime :: Int64
    num_propagation :: Int64
    num_input_splits :: Int64
    num_neuron_splits :: Int64
    initial_δ_bound :: Float64
    final_δ_bound :: Float64
end

In [33]:
function read_results(csv_dir::String)
    results = Result[]
    open(csv_dir, "r") do f
        while !eof(f)
            line = split(readline(f), ",")
            net = replace(line[1], " " => "")
            spec = replace(line[2], " " => "")
            status = replace(line[3], " " => "")
            status = status == "SAFE" ? VeryDiff.SAFE : (status == "UNSAFE" ? VeryDiff.UNSAFE : VeryDiff.UNKNOWN)
            runtime = parse(Int64, string(line[4]))
            num_propagation = parse(Int64, string(line[5]))
            num_input_splits = parse(Int64, string(line[6]))
            num_neuron_splits = parse(Int64, string(line[7]))
            initial_δ_bound = parse(Float64, string(line[8]))
            final_δ_bound = parse(Float64, string(line[9]))
            push!(results, Result(net, spec, status, runtime, num_propagation, num_input_splits, num_neuron_splits, initial_δ_bound, final_δ_bound))
        end
    end
    return results
end

read_results (generic function with 1 method)

In [34]:
cur_dir = @__DIR__
exp_final_dir = "$cur_dir\\experiments_final"

verydiff_exp_name = "VeryDiff"
deepsplit_exp_name = "DeepSplit-Base"
deepsplit_alt_exp_name = "DeepSplit-Alt"
deepsplit_input_exp_name = "DeepSplit-Base-Input"
deepsplit_diffzono_exp_name = "DeepSplit-Base-DiffZono"
deepsplit_input_diffzono_exp_name = "DeepSplit-Base-Input-DiffZono"

acas_results = "acas-0.5\\results.csv"
mnist_results = "mnist-1.0\\results.csv"

verydiff_acas_csv_dir = joinpath(exp_final_dir, verydiff_exp_name, acas_results)
deepsplit_acas_csv_dir = joinpath(exp_final_dir, deepsplit_exp_name, acas_results)
deepsplit_alt_acas_csv_dir = joinpath(exp_final_dir, deepsplit_alt_exp_name, acas_results)
deepsplit_input_acas_csv_dir = joinpath(exp_final_dir, deepsplit_input_exp_name, acas_results)
deepsplit_diffzono_acas_csv_dir = joinpath(exp_final_dir, deepsplit_diffzono_exp_name, acas_results)
deepsplit_input_diffzono_acas_csv_dir = joinpath(exp_final_dir, deepsplit_input_diffzono_exp_name, acas_results)

verydiff_mnist_csv_dir = joinpath(exp_final_dir, verydiff_exp_name, mnist_results)
deepsplit_mnist_csv_dir = joinpath(exp_final_dir, deepsplit_exp_name, mnist_results)
deepsplit_alt_mnist_csv_dir = joinpath(exp_final_dir, deepsplit_alt_exp_name, mnist_results)
deepsplit_input_mnist_csv_dir = joinpath(exp_final_dir, deepsplit_input_exp_name, mnist_results)
deepsplit_diffzono_mnist_csv_dir = joinpath(exp_final_dir, deepsplit_diffzono_exp_name, mnist_results)
deepsplit_input_diffzono_mnist_csv_dir = joinpath(exp_final_dir, deepsplit_input_diffzono_exp_name, mnist_results)

println("...")

...


In [35]:
sort_func = res -> res.runtime

verydiff_acas_results = sort!(read_results(verydiff_acas_csv_dir), by=sort_func)
deepsplit_acas_results = sort!(read_results(deepsplit_acas_csv_dir), by=sort_func)
deepsplit_alt_acas_results = sort!(read_results(deepsplit_alt_acas_csv_dir), by=sort_func)
deepsplit_input_acas_results = sort!(read_results(deepsplit_input_acas_csv_dir), by=sort_func)
deepsplit_diffzono_acas_results = sort!(read_results(deepsplit_diffzono_acas_csv_dir), by=sort_func)
deepsplit_input_diffzono_acas_results = sort!(read_results(deepsplit_input_diffzono_acas_csv_dir), by=sort_func)

verydiff_mnist_results = sort!(read_results(verydiff_mnist_csv_dir), by=sort_func)
deepsplit_mnist_results = sort!(read_results(deepsplit_mnist_csv_dir), by=sort_func)
deepsplit_alt_mnist_results = sort!(read_results(deepsplit_alt_mnist_csv_dir), by=sort_func)
deepsplit_input_mnist_results = sort!(read_results(deepsplit_input_mnist_csv_dir), by=sort_func)
deepsplit_diffzono_mnist_results = sort!(read_results(deepsplit_diffzono_mnist_csv_dir), by=sort_func)
deepsplit_input_diffzono_mnist_results = sort!(read_results(deepsplit_input_diffzono_mnist_csv_dir), by=sort_func)

all_acas_results = [verydiff_acas_results, deepsplit_acas_results, deepsplit_alt_acas_results, deepsplit_input_acas_results, deepsplit_diffzono_acas_results, deepsplit_input_diffzono_acas_results]
all_mnist_results = [verydiff_mnist_results, deepsplit_mnist_results, deepsplit_alt_mnist_results, deepsplit_input_mnist_results, deepsplit_diffzono_mnist_results, deepsplit_input_diffzono_mnist_results]

6-element Vector{Vector{Result}}:
 [Result("mnist_relu_3_100_0.79_adam_1_stepsize", "mnist_0_local_15", VeryDiff.SAFE, 1378444, 1, 0, 0, 0.9762136532998933, 0.9762136532998933), Result("mnist_relu_3_100_0.79_adam_1_stepsize", "mnist_10_local_15", VeryDiff.SAFE, 1390721, 1, 0, 0, 0.6017052510644736, 0.6017052510644736), Result("mnist_relu_3_100_0.79_adam_1_stepsize", "mnist_1_local_15", VeryDiff.SAFE, 1406744, 1, 0, 0, 0.9269471996009466, 0.9269471996009466), Result("mnist_relu_3_100_0.79_adam_1_stepsize", "mnist_4_local_15", VeryDiff.SAFE, 1435391, 1, 0, 0, 0.839842867068394, 0.839842867068394), Result("mnist_relu_3_100_0.79_adam_1_stepsize", "mnist_8_local_15", VeryDiff.SAFE, 1448600, 1, 0, 0, 0.9878349475800786, 0.9878349475800786), Result("mnist_relu_3_100_0.79_adam_1_stepsize", "mnist_9_local_15", VeryDiff.SAFE, 1464628, 1, 0, 0, 0.9476254177893607, 0.9476254177893607), Result("mnist_relu_3_100_0.79_adam_1_stepsize", "mnist_5_local_15", VeryDiff.SAFE, 4250638, 3, 1, 0, 1.0041987865

In [36]:
function count_solved(results::Vector{Result})
    count(r -> r.status != VeryDiff.UNKNOWN, results)
end

count_solved (generic function with 1 method)

In [37]:
x_acas = [1:count_solved(results) for results in all_acas_results]
y_acas = [[results[k].runtime / 1e9 for k in x_acas[i]] for (i, results) in enumerate(all_acas_results)]

6-element Vector{Vector{Float64}}:
 [0.000483285, 0.000513286, 0.000612309, 0.001198747, 0.001357577, 0.001401937, 0.001454556, 0.001456381, 0.001491398, 0.001529389  …  5.465051756, 7.579443666, 7.594409729, 8.264870775, 8.530710178, 9.05766453, 9.261310511, 9.435128518, 9.988624942, 13.862977662]
 [0.00049026, 0.000507911, 0.000758987, 0.001083083, 0.001140711, 0.001263347, 0.001342369, 0.001437841, 0.001450221, 0.001456764  …  0.002600382, 0.013181515, 0.015258433, 0.117770397, 0.157245569, 0.174232404, 0.906424509, 1.975996233, 4.56518703, 116.885871935]
 [0.000415317, 0.0004369, 0.000452795, 0.0011064, 0.001109351, 0.001169736, 0.001296943, 0.001327802, 0.001354314, 0.001462954  …  0.001659251, 0.002093318, 0.002177528, 0.130100339, 0.15741398, 0.16523416, 0.922911619, 1.987540257, 4.581955606, 116.463796039]
 [0.000391467, 0.000455443, 0.000501573, 0.001053097, 0.0011658, 0.001260351, 0.001394274, 0.001409246, 0.001461428, 0.00147348  …  8.481328218, 8.489300923, 9.043449796, 12.

In [38]:
# linewidth = 2.0
plot(x_acas[1], y_acas[1], label="VeryDiff", dpi=400, title="ACAS Benchmarks", xlabel="Number of Instances Verified", ylabel="Time (s)", legend=:legend, framestyle=:origin)
plot!(x_acas[2], y_acas[2], label="DeepSplit-Base")
plot!(x_acas[3], y_acas[3], label="DeepSplit-Alt")
plot!(x_acas[4], y_acas[4], label="DeepSplit-Base-Input")
plot!(x_acas[5], y_acas[5], label="DeepSplit-Base-DiffZono")
plot!(x_acas[6], y_acas[6], label="DeepSplit-Base-Input-DiffZono")

savefig("acas_benchmarks.png")

"c:\\Users\\jakdar\\uni\\bachelorarbeit\\VeryDiff\\dev\\experiments\\acas_benchmarks.png"

In [39]:
x_mnist = [1:count_solved(results) for results in all_mnist_results]
y_mnist = [[results[k].runtime / 1e9 for k in x_mnist[i]] for (i, results) in enumerate(all_mnist_results)]

6-element Vector{Vector{Float64}}:
 [0.001378444, 0.001390721, 0.001406744, 0.001435391, 0.0014486, 0.001464628, 0.004250638, 0.008880225, 0.019297298, 0.025707291  …  0.632306755, 1.343554582, 1.675129774, 2.90971231, 3.509331593, 4.14777903, 4.310045306, 7.243081671, 10.514891249, 14.519371999]
 [0.001321383, 0.001337689, 0.001337743, 0.001388654, 0.001395223, 0.001743182, 0.01096307, 0.015994952, 0.017196876, 0.017225974  …  1.111158915, 1.303816637, 8.315101902, 10.200323858, 11.529918421, 25.255122155, 38.196841067, 42.256306243, 58.053966498, 62.214939992]
 [0.00130822, 0.001328432, 0.00134522, 0.001356126, 0.001373644, 0.00145005, 0.010913297, 0.01611097, 0.017161093, 0.017161108  …  1.132084046, 1.305980554, 7.814088894, 10.231877362, 11.59341381, 25.125867309, 38.121941123, 42.059183983, 57.518048118, 61.813550388]
 [0.001231715, 0.001290847, 0.001346112, 0.001347237, 0.001367323, 0.001437351, 0.003889933, 0.009515902, 0.017115266, 0.017269599  …  4.488264784, 5.106038079, 8.4

In [40]:
plot(x_mnist[1], y_mnist[1], label="VeryDiff", dpi=400, title="MNIST Benchmarks", xlabel="Number of Instances Verified", ylabel="Time (s)", legend=:legend, framestyle=:origin)
plot!(x_mnist[2], y_mnist[2], label="DeepSplit-Base")
plot!(x_mnist[3], y_mnist[3], label="DeepSplit-Alt")
plot!(x_mnist[4], y_mnist[4], label="DeepSplit-Base-Input")
plot!(x_mnist[5], y_mnist[5], label="DeepSplit-Base-DiffZono")
plot!(x_mnist[6], y_mnist[6], label="DeepSplit-Base-Input-DiffZono")

savefig("mnist_benchmarks.png")

"c:\\Users\\jakdar\\uni\\bachelorarbeit\\VeryDiff\\dev\\experiments\\mnist_benchmarks.png"